# Fetching Pre and Post synaptic Neuron Location

## Libraries

In [1]:
# ! pip install standard-transform
# ! pip install --upgrade caveclient

In [2]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from microns_phase3 import nda
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


## Preparation

Function that transforms the voxel coorindate into a depth coordinate (function given to us by the authors of the Microns Project)

In [3]:
t_form_v = minnie_transform_vx()

### Querying the data from CAVEclient

In [4]:
client = CAVEclient()

In [5]:
# client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

We will use the latest version of the minnie65 dataset, i.e. 661

In [6]:
client = CAVEclient('minnie65_public_v661')

In [7]:
client.materialize.get_tables()

['baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117']

These are all the available tables, but we are interested only in the *apl_functional_coreg_forward_v5*, that is the one containing the information about the proofread neurons

In [35]:
temp.head()

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
0,1,2022-12-16 00:55:27.003407+00:00,NaN,t,connectivity_group,19,88470829612207965,864691135511632080,"[172192, 189648, 21784]"
1,2,2022-12-16 00:55:27.004309+00:00,NaN,t,connectivity_group,12,89880747250804048,864691136723962621,"[182384, 208608, 22961]"
2,3,2022-12-16 00:55:27.005162+00:00,NaN,t,connectivity_group,4,91137763852205616,864691136371491976,"[191712, 136928, 22487]"
3,4,2022-12-16 00:55:27.006024+00:00,NaN,t,connectivity_group,6,90079140245935801,864691135777733216,"[184208, 114224, 21914]"
4,5,2022-12-16 00:55:27.006859+00:00,NaN,t,connectivity_group,2,89589376669051726,864691135490940135,"[180752, 135200, 22612]"


In [10]:
df = client.materialize.query_table('apl_functional_coreg_forward_v5')

We start by setting as the index he ID associated to the root of the synapse

In [11]:
df = df.set_index('pt_root_id')

In [12]:
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
pt_root_id,,,,,,,,,,,,,,,,,,
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,67007,2023-04-06 15:05:50.491194+00:00,t,255686,9,6,1913,2,8.941486,4.206369,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,65810,2023-04-06 15:05:49.502144+00:00,t,255686,9,4,7548,6,9.380808,-5.165012,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,54982,2023-04-06 15:05:15.938520+00:00,t,255686,8,7,4778,4,3.763804,5.341737,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,46412,2023-04-06 15:04:43.197444+00:00,t,255686,7,5,1946,2,4.071072,3.306988,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,11823,2023-04-06 15:03:01.038836+00:00,t,255686,5,3,1994,2,5.419972,6.644126,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"


In [13]:
df.shape

(68873, 18)

In [14]:
df.shape[0]//5000, df.shape[0]%5000

(13, 3873)

In [15]:
unit_keys_1 = df.iloc[:5000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
# unit_keys_2 = df.iloc[5000:10000, ]

In [27]:
area_membership = []
for i in range(13):
    unit_keys = df.iloc[i*5000:i*5000+5000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
    temp = (nda.AreaMembership & unit_keys).fetch('brain_area')
    area_membership = np.concatenate((area_membership, temp), axis=0)
    print('Step' + i + 'completed!')
unit_keys = df.iloc[65000: 68873, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
temp = (nda.AreaMembership & unit_keys).fetch('brain_area')
area_membership = np.concatenate((area_membership, temp), axis=0)

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


0
1
2
3
4
5
6
7
8
9
10
11
12


In [32]:
df['Area Membership'] = area_membership
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position,Area Membership
pt_root_id,,,,,,,,,,,,,,,,,,,
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,67007,2023-04-06 15:05:50.491194+00:00,t,255686,9,6,1913,2,8.941486,4.206369,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,65810,2023-04-06 15:05:49.502144+00:00,t,255686,9,4,7548,6,9.380808,-5.165012,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,54982,2023-04-06 15:05:15.938520+00:00,t,255686,8,7,4778,4,3.763804,5.341737,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,46412,2023-04-06 15:04:43.197444+00:00,t,255686,7,5,1946,2,4.071072,3.306988,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135568539372,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,11823,2023-04-06 15:03:01.038836+00:00,t,255686,5,3,1994,2,5.419972,6.644126,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]",V1


In [33]:
input_dfs = [client.materialize.synapse_query(post_ids=864691135568539372),
             client.materialize.synapse_query(post_ids=864691135568539372)]
input_dfs


HTTPError: 500 Server Error: 404 Not Found: analysis table synapses_pni_2 not found for version 661 in datastack minnie65_public_v661 for url: https://minnie.microns-daf.com/materialize/api/v3/datastack/minnie65_public_v661/version/661/table/synapses_pni_2/query?return_pyarrow=True&arrow_format=True&split_positions=True content:b'{"code": 500, "message": "404 Not Found: analysis table synapses_pni_2 not found for version 661 in datastack minnie65_public_v661", "traceback": ["Traceback (most recent call last):\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask/app.py\\", line 1516, in full_dispatch_request\\n    rv = self.dispatch_request()\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask/app.py\\", line 1502, in dispatch_request\\n    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)\\n", "  File \\"/usr/local/lib/python3.7/site-packages/flask_restx/api.py\\", line 403, in wrapper\\n    resp = resource(*args, **kwargs)\\n", "  File \\"/u